In [4]:
import random

from matplotlib import pyplot as plt
import numpy as np

from arch import arch_model

from Garch import GARCH

### 1. Generate synthetic data for basic GARCH(1,1)

Mean Model
$$ r_t = e_t $$ 
$$e_t \sim N(0, \sigma_t^2)$$

Volatility Model 
$$ \sigma_t^2 = \omega +  \alpha e_{t-1}^2 + \beta \sigma_{t-i}^2  $$

In [2]:
e = [0]  # errors e_t
sigma2 = [1] # sigma sigma_t
omega, alpha, beta = 0.2, 0.3, 0.5

for t in range(1000):
    sigma2_t = omega + alpha * e[-1]**2 + beta * sigma2[-1]
    e_t = random.gauss(0, sigma2_t**0.5)
    
    e.append(e_t)
    sigma2.append(sigma2_t)

e = e[1:]
sigma2 = sigma2[1:]

In [3]:
# Fit using ARCH library
model = arch_model(100*e, vol='GARCH', mean='zero', p=1, q=1)
garch_fit = model.fit(disp='off')

garch_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                       Zero Mean - GARCH Model Results                        
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -128924.
Distribution:                  Normal   AIC:                           257855.
Method:            Maximum Likelihood   BIC:                           257883.
                                        No. Observations:               100000
Date:                Tue, Feb 04 2025   Df Residuals:                   100000
Time:                        21:41:04   Df Model:                            0
                            Volatility Model                            
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
omega          0.1524  4.350e-03     35.033 6.978e-269 [  0.144,  0.161]
alpha[1]       0.2135  4.325e-03     49.352      0.000 [  0.205,  0.222]
beta[1]        0.6094  7.794e-03     78.188      0.000 [  0.594,  0.625]
========================================================================

Covariance estimator: robust
"""

### 2. Generate synthetic data for GARCH(1,1) with one exo variable in the volatility model

* Assumption: exo variable is normally distributed with $ X \sim N(0, 1) $

Mean Model
$$ r_t = e_t $$ 
$$e_t \sim N(0, \sigma_t^2)$$

Volatility Model 
$$ \sigma_t^2 = \omega +  \alpha e_{t-1}^2 + \beta \sigma_{t-i}^2 + 
    \gamma x_t$$

$$x_t \sim N(0, 1)$$

In [9]:
# Define volatility component coefficients
omega, alpha, beta, gamma = 0.1, 0.1, 0.4, 0.05

T = 1000

In [6]:
e = [0]  # errors e_t
sigma2 = [1] # sigma sigma_t
x = np.random.randn(T, 1)

for t in range(T):
    sigma2_t = omega + alpha * e[-1]**2 + beta * sigma2[-1] + gamma * x[t]**2
    e_t = random.gauss(0, sigma2_t**0.5) 

    e.append(e_t)
    sigma2.append(sigma2_t)

e = e[1:]
sigma2 = sigma2[1:]

In [8]:
# Fit using ARCH library, without exogeneous
model = arch_model(e, vol='GARCH', mean='zero', p=1, q=1)
garch_fit = model.fit(disp='off')

garch_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                       Zero Mean - GARCH Model Results                        
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.001
Vol Model:                      GARCH   Log-Likelihood:               -820.942
Distribution:                  Normal   AIC:                           1647.88
Method:            Maximum Likelihood   BIC:                           1662.61
                                        No. Observations:                 1000
Date:                Tue, Feb 04 2025   Df Residuals:                     1000
Time:                        21:43:25   Df Model:                            0
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.2241  5.002e-02      4.479  7.484e-06   [  0.126,  0.322]
alpha[1]       0.1700  4.583e-02      3.709  2.083e-04 [8.014e-02,  0.260]
beta[1]        0.1061      0.168      0.631      0.528   [ -0.223,  0.436]
==========================================================================

Covariance estimator: robust
"""

In [10]:
# Fit without exogenous
garch_without_exo = GARCH(p=1, q=1, z=0)
garch_without_exo.train(e)

optimising
optimising finished
omega=0.22167227843418752
 alpha=array([0.17082962])
 beta=array([0.1131281])



In [13]:
garch_with_exo = GARCH(p=1, q=1, z=1)
garch_with_exo.train(e, x=x)

optimising
optimising finished
omega=0.1089387896364006
 alpha=array([0.1250687])
 beta=array([0.36885773])
 gammas=array([[0.04418365]])
